In [1]:
!pip install rdflib
!pip install pyshacl

In [2]:
import rdflib
import time
start = time.process_time()

bimGraph = rdflib.Graph()
result = bimGraph.parse('bimreq.ttl', format='ttl') 
result = bimGraph.parse('muster_house_05_20210813__IFC4.ttl', format='ttl') 

print(f'graph loaded in {int(time.process_time() - start)} s')

graph loaded in 44 s


In [ ]:
from pyshacl import validate

rules = '''
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix express: <https://w3id.org/express#> .
@prefix ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC2x3/TC1/OWL#> .

@prefix : <http://demo.dsd.sztaki.hu/bimreqShacl#> .

:req02Shape a sh:NodeShape ;
            sh:property [
                sh:path ( ifc:overallHeight_IfcWindow express:hasDouble ) ;
                sh:maxInclusive "1.3"^^xsd:double ;
                sh:message "Windows should not be higher than 1.3"@en ;
                sh:severity sh:Warning
            ] ;
            sh:targetClass ifc:IfcWindow ;
.

'''

start = time.process_time()

shaclGraph = rdflib.Graph()
shaclGraph = shaclGraph.parse(data=rules, format="ttl")

r = validate(bimGraph,
      shacl_graph=shaclGraph,
      ont_graph=None,
      inference='rdfs',
      abort_on_first=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
conforms, results_graph, results_text = r
print(f'shacl run in {int(time.process_time() - start)} s')
print(results_text)

In [40]:
query = '''
    #prefix ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC2x3/TC1/OWL#>
    #prefix ifc: <http://www.buildingsmart-tech.org/ifcOWL/IFC2X3_TC1#>
    prefix ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#>
    prefix express: <https://w3id.org/express#>

    SELECT *
        WHERE {
          {
            ?propSet ifc:relatedObjects_IfcRelDefinesByProperties ?object.
            ?object rdf:type ifc:IfcWall.
            ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
            ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasString ?value.
            ?propSv ifc:name_IfcProperty/express:hasString  "AcousticRating".
            FILTER (?value >= "56")
          } 
        }
'''
''' {
            ?propSet ifc:relatedObjects_IfcRelDefinesByProperties ?object.
            ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
            ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasDouble ?value.
            ?propSv ifc:name_IfcProperty/express:hasString  "ThermalTransmittance".        
          }
'''
#             ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasDouble ?value.
#            ?propSv ifc:name_IfcProperty/express:hasString  "ThermalTransmittance".
# or 
# ?measure rdf:type ifc:IfcThermalTransmittanceMeasure.
print(f"Graph has {len(bimGraph)} statements.")
qres = bimGraph.query(query)
print(f"{len(qres)} query results.")
for row in qres:
    #print(row)
    print(f'{row.object} {row.value}')

Graph has 309348 statements.
22 query results.
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_3418 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_43630 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_22049 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_3523 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_33193 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_43740 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_19290 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_19185 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_43305 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_43410 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_17876 57
http://linkedbuildingdata.net/ifc/resources20210904_180957/IfcWall_27645 59
http://linkedbuildingdata.net/ifc/resources

In [28]:
from pyshacl import validate

rules = '''
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix express: <https://w3id.org/express#> .
@prefix ifcowl: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#> .

@prefix : <http://demo.dsd.sztaki.hu/bimreqShacl#> .

:req03Shape a sh:NodeShape ;
    sh:targetClass ifcowl:IfcWindow ;
    sh:message "Windows U-value should not be higher than 1.15."@en;
    sh:sparql [
        a sh:SPARQLConstraint ;
        sh:select """
            PREFIX ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#>
            PREFIX express: <https://w3id.org/express#>

            SELECT $this ?value
                WHERE {
                    ?propSet ifc:relatedObjects_IfcRelDefinesByProperties $this.
                    ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
                    ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasDouble ?value.
                    ?propSv ifc:name_IfcProperty/express:hasString  "ThermalTransmittance".
                    FILTER (?value > 1.15)
                }
        """ ;
    ]
.

'''
shaclGraph = rdflib.Graph()
shaclGraph = shaclGraph.parse(data=rules, format="ttl")

r = validate(bimGraph,
      shacl_graph=shaclGraph,
      ont_graph=None,
      inference='rdfs',
      abort_on_first=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
conforms, results_graph, results_text = r
print(results_text)

Validation Report
Conforms: False
Results (16):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :req03Shape
	Focus Node: inst:IfcWindow_37426
	Value Node: Literal("6.7069", datatype=xsd:double)
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:select Literal("
            PREFIX ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#>
            PREFIX express: <https://w3id.org/express#>

            SELECT $this ?value
                WHERE {
                    ?propSet ifc:relatedObjects_IfcRelDefinesByProperties $this.
                    ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
                    ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasDouble ?value.
                    ?propSv ifc:name_IfcProperty/express:hasString  "ThermalTransmittance".
                    FILTER (?value > 1.1

In [41]:
from pyshacl import validate

rules = '''
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix express: <https://w3id.org/express#> .
@prefix ifcowl: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#> .

@prefix : <http://demo.dsd.sztaki.hu/bimreqShacl#> .

:req03Shape a sh:NodeShape ;
    sh:targetClass ifcowl:IfcWall ;
    sh:message "Airborne sound insulation value should be higher than 56dB"@en;
    sh:sparql [
        a sh:SPARQLConstraint ;
        sh:select """
            PREFIX ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#>
            PREFIX express: <https://w3id.org/express#>

            SELECT $this ?value
                WHERE {
                    $this rdf:type ifc:IfcWall.
                    ?propSet ifc:relatedObjects_IfcRelDefinesByProperties $this.
                    ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
                    ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasString ?value.
                    ?propSv ifc:name_IfcProperty/express:hasString  "AcousticRating".
                    FILTER (?value >= "56")
                }
        """ ;
    ]
.

'''

shaclGraph = rdflib.Graph()
shaclGraph = shaclGraph.parse(data=rules, format="ttl")

r = validate(bimGraph,
      shacl_graph=shaclGraph,
      ont_graph=None,
      inference='rdfs',
      abort_on_first=False,
      allow_warnings=False,
      meta_shacl=False,
      advanced=False,
      js=False,
      debug=False)
conforms, results_graph, results_text = r
print(results_text)

Validation Report
Conforms: False
Results (22):
Constraint Violation in SPARQLConstraintComponent (http://www.w3.org/ns/shacl#SPARQLConstraintComponent):
	Severity: sh:Violation
	Source Shape: :req03Shape
	Focus Node: inst:IfcWall_27645
	Value Node: Literal("59")
	Source Constraint: [ rdf:type sh:SPARQLConstraint ; sh:select Literal("
            PREFIX ifc: <http://standards.buildingsmart.org/IFC/DEV/IFC4/ADD2_TC1/OWL#>
            PREFIX express: <https://w3id.org/express#>

            SELECT $this ?value
                WHERE {
                    $this rdf:type ifc:IfcWall.
                    ?propSet ifc:relatedObjects_IfcRelDefinesByProperties $this.
                    ?propSet ifc:relatingPropertyDefinition_IfcRelDefinesByProperties/ifc:hasProperties_IfcPropertySet ?propSv.
                    ?propSv ifc:nominalValue_IfcPropertySingleValue/express:hasString ?value.
                    ?propSv ifc:name_IfcProperty/express:hasString  "AcousticRating".
                    FILTE